<a href="https://colab.research.google.com/github/wiiiiish/2022-machine_learning/blob/main/time_series_0507.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [3]:
%cd drive/MyDrive/Colab\ Notebooks/2022-04-27\ 데이터

/content/drive/MyDrive/Colab Notebooks/2022-04-27 데이터


In [22]:
import tensorflow as tf
tf.__version__

'2.8.0'

In [24]:
# import library
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow

from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

# Set seeds to make the experiment more reprodecible
from tensorflow.compat.v1 import set_random_seed
from numpy.random import seed
set_random_seed(1)
seed(1)

In [46]:
# data
aod_data = pd.read_csv('./data/2020-05-03_aod.csv', encoding='utf-8')
aod_data

,date_time,aod
0,2020-05-03 00:00:00,0.0
1,2020-05-03 00:00:10,0.0
2,2020-05-03 00:00:20,0.0
3,2020-05-03 00:00:30,0.0
4,2020-05-03 00:00:40,0.0
...,...,...
155516,2020-05-20 23:59:20,0.0
155517,2020-05-20 23:59:30,0.0
155518,2020-05-20 23:59:40,0.0
155519,2020-05-20 23:59:50,0.0


In [47]:
# 시간 데이터 날짜 바꾸기
for i in range(155521):
  new_day = (int)(aod_data['date_time'][i][8:10]) + 2
  if new_day < 10:
    new_day = f"0{new_day}"
  else:
    new_day = f"{new_day}"
  
  timeList = list(aod_data['date_time'][i])
  timeList[8:10] = new_day
  new_date = ''.join(s for s in timeList)

  aod_data['date_time'][i] = new_date

aod_data

,date_time,aod
0,2020-05-05 00:00:00,0.0
1,2020-05-05 00:00:10,0.0
2,2020-05-05 00:00:20,0.0
3,2020-05-05 00:00:30,0.0
4,2020-05-05 00:00:40,0.0
...,...,...
155516,2020-05-22 23:59:20,0.0
155517,2020-05-22 23:59:30,0.0
155518,2020-05-22 23:59:40,0.0
155519,2020-05-22 23:59:50,0.0


In [48]:
# dataframe -> csv
aod_data.to_csv('./data/0505-0523_aod.csv')

In [55]:
aod_0505 = aod_data.iloc[:8640,:]
aod_0505.to_csv('./data/0505_aod')

In [56]:
weather_data = pd.read_csv("./data/weather_data.csv", encoding='utf-8')
weather_data

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
0,05-01 00:00:00,19.471425,0.000000,1.283509,99.085068,0.0,-0.571956,2000.060648,20.859082
1,05-01 00:00:10,19.515992,0.000000,1.288677,98.906960,0.0,3.277923,2000.042480,20.930770
2,05-01 00:00:20,19.456078,0.000000,1.053963,98.952560,0.0,0.779748,2000.018245,20.992075
3,05-01 00:00:30,19.369269,0.000000,1.215434,98.990665,0.0,0.660592,1999.990462,20.983526
4,05-01 00:00:40,19.518434,0.000000,1.225970,99.000163,0.0,4.247148,2000.043941,20.894728
...,...,...,...,...,...,...,...,...,...
527035,06-30 23:59:10,25.494836,0.237966,0.546747,90.711643,0.0,9.445173,0.000000,27.381028
527036,06-30 23:59:20,25.428284,0.278660,0.675943,90.426103,0.0,9.788200,0.000000,27.304989
527037,06-30 23:59:30,25.400970,0.161461,0.643305,90.306220,0.0,8.804747,0.000000,27.346739
527038,06-30 23:59:40,25.503257,0.083391,0.789334,90.616037,0.0,9.496567,0.000000,27.327265


In [65]:
weather_0505 = weather_data.iloc[34560:43200,:]
weather_0505

,date_time,temperature,precipitation,wind_speed,humidity,radiation,total_cloudiness,visibility,ground_temp
34560,05-05 00:00:00,20.561767,0.0,0.749072,98.937978,0.0,4.624748,2000.017467,21.489114
34561,05-05 00:00:10,20.569543,0.0,0.470559,98.978541,0.0,1.546086,1999.916159,21.535559
34562,05-05 00:00:20,20.552419,0.0,0.703899,98.920235,0.0,4.227162,2000.068799,21.542335
34563,05-05 00:00:30,20.514595,0.0,0.612151,98.944682,0.0,0.331357,2000.010445,21.525727
34564,05-05 00:00:40,20.556292,0.0,0.676211,99.041025,0.0,3.207824,2000.004595,21.563515
...,...,...,...,...,...,...,...,...,...
43195,05-05 23:59:10,20.823070,0.0,0.900223,99.083700,0.0,1.462444,941.224556,21.147663
43196,05-05 23:59:20,20.629803,0.0,0.901650,99.079732,0.0,1.555491,1180.409768,21.065588
43197,05-05 23:59:30,20.639215,0.0,0.908953,98.977158,0.0,0.443561,933.458536,21.096623
43198,05-05 23:59:40,20.788708,0.0,0.899667,99.015539,0.0,0.437883,985.787304,21.205976


In [68]:
weather_0505.to_csv('./data/weather_0505.csv')